<a href="https://colab.research.google.com/github/bangjaia/Keras/blob/main/%ED%95%84%EA%B8%B0%EC%B2%B4_%EC%83%9D%EC%84%B1_%ED%95%A9%EC%84%B1%EA%B3%B1_%EA%B3%84%EC%B8%B5_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
################################
# 공통 패키지 불러오기
################################
import numpy as np
from PIL import Image
import math
import os

from keras import models, layers, optimizers
from keras.datasets import mnist
import keras.backend as K

print(K.image_data_format)

import tensorflow as tf

def mse_4d(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true), axis=(1,2,3))

def mse_4d_tf(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_pred - y_true), axis=(1,2,3))

################################
# GAN 모델링
################################
class GAN(models.Sequential):
    def __init__(self, input_dim=62): # input_dim = args.n_train = 32
        """
        self, self.generator, self.discriminator are all models
        """
        super().__init__()
        self.input_dim = input_dim

        self.generator = self.GENERATOR()
        self.discriminator = self.DISCRIMINATOR()
        self.add(self.generator)
        self.discriminator.trainable = False
        self.add(self.discriminator)

        self.compile_all()

    def compile_all(self):
        # Compiling stage
        """
        lr의 경우 Keras3 이상 버전부터 사용되지 않고 대신 learning_rate가 사용됨.

        그 외의 변경된 문법
        lr	                            learning_rate
        decay	                          weight_decay
        keras.optimizers.Adam(lr=...)	  keras.optimizers.Adam(learning_rate=...)
        model.fit(x, y, nb_epoch=...)	  model.fit(x, y, epochs=...)
        현재코드에선 lr만 변경
        """
        d_optim = optimizers.SGD(learning_rate=0.0005, momentum=0.9, nesterov=True)
        g_optim = optimizers.SGD(learning_rate=0.0005, momentum=0.9, nesterov=True)
        self.generator.compile(loss=mse_4d_tf, optimizer="SGD")
        self.compile(loss='binary_crossentropy', optimizer=g_optim)
        self.discriminator.trainable = True
        self.discriminator.compile(loss='binary_crossentropy', optimizer=d_optim)

    def GENERATOR(self):
        input_dim = self.input_dim

        model = models.Sequential()
        model.add(layers.Dense(1024, activation='tanh', input_dim=input_dim))
        model.add(layers.Dense(7 * 7 * 128, activation='tanh')) # H, W, C = 7, 7, 128
        model.add(layers.BatchNormalization())
        # The Conv2D op currently only supports the NHWC tensor format on the CPU.
        model.add(layers.Reshape((7, 7, 128), input_shape=(7 * 7 * 128,)))
        model.add(layers.UpSampling2D(size=(2, 2)))
        model.add(layers.Conv2D(64, (5, 5), padding='same', activation='tanh'))
        model.add(layers.UpSampling2D(size=(2, 2)))
        model.add(layers.Conv2D(1, (5, 5), padding='same', activation='tanh'))
        return model

    def DISCRIMINATOR(self):
        # The Conv2D op currently only supports the NHWC tensor format on the CPU.
        model = models.Sequential()
        model.add(layers.Conv2D(64, (5, 5), padding='same', activation='tanh',
                                input_shape=(28, 28, 1)))
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(layers.Conv2D(128, (5, 5), activation='tanh'))
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(layers.Flatten())
        model.add(layers.Dense(1024, activation='tanh'))
        model.add(layers.Dense(1, activation='sigmoid'))
        return model

    def get_z(self, ln):
        input_dim = self.input_dim
        return np.random.uniform(-1, 1, (ln, input_dim))

    def train_both(self, x):
        ln = x.shape[0]
        # First trial for training discriminator
        z = self.get_z(ln)
        w = self.generator.predict(z, verbose=0)
        xw = np.concatenate((x, w))
        y2 = np.array([1] * ln + [0] * ln).reshape(-1,1) # Necessary!
        d_loss = self.discriminator.train_on_batch(xw, y2)

        # Second trial for training generator
        z = self.get_z(ln)
        self.discriminator.trainable = False
        g_loss = self.train_on_batch(z, np.array([1] * ln).reshape(-1, 1))
        self.discriminator.trainable = True

        return d_loss, g_loss

################################
# GAN 학습하기
################################
def combine_images(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num) / width))
    shape = generated_images.shape[1:3] # (1,2) for NHWC
    image = np.zeros((height * shape[0], width * shape[1]),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index / width)
        j = index % width
        image[i * shape[0]:(i + 1) * shape[0],
        j * shape[1]:(j + 1) * shape[1]] = img[ :, :, 0] # NHWC
    return image

def get_x(X_train, index, BATCH_SIZE):
    return X_train[index * BATCH_SIZE:(index + 1) * BATCH_SIZE]

def save_images(generated_images, output_fold, epoch, index):
    image = combine_images(generated_images)
    image = image * 127.5 + 127.5
    Image.fromarray(image.astype(np.uint8)).save(
        output_fold + '/' +
        str(epoch) + "_" + str(index) + ".png")

def load_data(n_train):
    (X_train, y_train), (_, _) = mnist.load_data()
    return X_train[:n_train]

def train(args):
    BATCH_SIZE = args.batch_size
    epochs = args.epochs
    output_fold = args.output_fold
    input_dim = args.input_dim
    n_train = args.n_train

    os.makedirs(output_fold, exist_ok=True)
    print('Output_fold is', output_fold)

    X_train = load_data(n_train)

    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    # The Conv2D op currently only supports the NHWC tensor format on the CPU. The op was given the format: NCHW
    # X_train = X_train.reshape((X_train.shape[0], 1) + X_train.shape[1:]) # <-- NCHW format
    X_train = X_train.reshape(X_train.shape + (1,)) # <-- NHWC format

    gan = GAN(input_dim)

    d_loss_ll = []
    g_loss_ll = []
    for epoch in range(epochs):
        if epoch % 10 == 0:
            print("Epoch is", epoch)
            print("Number of batches", int(X_train.shape[0] / BATCH_SIZE))

        d_loss_l = []
        g_loss_l = []
        for index in range(int(X_train.shape[0] / BATCH_SIZE)):
            x = get_x(X_train, index, BATCH_SIZE)

            d_loss, g_loss = gan.train_both(x)

            d_loss_l.append(d_loss)
            g_loss_l.append(g_loss)

        if epoch % 10 == 0 or epoch == epochs - 1:
            z = gan.get_z(x.shape[0])
            w = gan.generator.predict(z, verbose=0)
            save_images(w, output_fold, epoch, 0)

        d_loss_ll.append(d_loss_l)
        g_loss_ll.append(g_loss_l)

    # gan.generator.save_weights(output_fold + '/' + 'generator', True)
    # gan.discriminator.save_weights(output_fold + '/' + 'discriminator', True)

    np.savetxt(output_fold + '/' + 'd_loss', d_loss_ll)
    np.savetxt(output_fold + '/' + 'g_loss', g_loss_ll)

################################
# GAN 예제 실행하기
################################
import argparse
import sys

def main():
    parser = argparse.ArgumentParser()
    """Jupyter에서 커널 실행시 자동으로 "-f" 옵션이 추가됨.
    argparse.ArgumentParser()에서는 해당 옵션 관련 처리가 없음.
    따라서 "-f"를 모르는 인자로 인식하여 에러 발생"""

    parser.add_argument('--batch_size', type=int, default=16,
        help='Batch size for the networks')
    parser.add_argument('--epochs', type=int, default=1000,
        help='Epochs for the networks')
    parser.add_argument('--output_fold', type=str, default='GAN_OUT',
        help='Output fold to save the results')
    parser.add_argument('--input_dim', type=int, default=10,
        help='Input dimension for the generator.')
    parser.add_argument('--n_train', type=int, default=32,
        help='The number of training data.')

    # args = parser.parse_args()
    args, _ = parser.parse_known_args(sys.argv[1:]) #위의 Jupyter 인자를 무시하는 코드. 인식 못한 인자를 무시

    train(args)


if __name__ == '__main__':
  main()

<function image_data_format at 0x7d29d4368220>
Output_fold is GAN_OUT
Epoch is 0
Number of batches 2
Epoch is 10
Number of batches 2
Epoch is 20
Number of batches 2
Epoch is 30
Number of batches 2
Epoch is 40
Number of batches 2
Epoch is 50
Number of batches 2
Epoch is 60
Number of batches 2
Epoch is 70
Number of batches 2
Epoch is 80
Number of batches 2
Epoch is 90
Number of batches 2
Epoch is 100
Number of batches 2
Epoch is 110
Number of batches 2
Epoch is 120
Number of batches 2
Epoch is 130
Number of batches 2
Epoch is 140
Number of batches 2
Epoch is 150
Number of batches 2
Epoch is 160
Number of batches 2
Epoch is 170
Number of batches 2
Epoch is 180
Number of batches 2
Epoch is 190
Number of batches 2
Epoch is 200
Number of batches 2
Epoch is 210
Number of batches 2
Epoch is 220
Number of batches 2
Epoch is 230
Number of batches 2
Epoch is 240
Number of batches 2
Epoch is 250
Number of batches 2
Epoch is 260
Number of batches 2
Epoch is 270
Number of batches 2
Epoch is 280
Numb